# Metryki w przestrzeni napisów

1. Zaimplementuj przynajmniej 3 metryki spośród wymienionych: cosinusowa, LCS, DICE, euklidesowa.
2. Zaimplementuj przynajmniej 2 sposoby oceny jakości klasteryzacji (np. indeks Daviesa-Bouldina).
3. Stwórz stoplistę najczęściej występujących słów.
4. Wykonaj klasteryzację zawartości załączonego pliku (lines.txt) przy użyciu przynajmniej 2 algorytmów oraz metryk zaimplementowanych w pkt. 1. i metryki Levenshteina. Każda linia to adres pocztowy firmy, różne sposoby zapisu tego samego adresu powinny się znaleźć w jednym klastrze.
5. Porównaj jakość wyników sposobami zaimplementowanymi w pkt. 2.
6. Czy masz jakiś pomysł na poprawę jakości klasteryzacji w tym zadaniu?

### Zad 1.

### LCS distance
$d(x,y) = 1-\frac{LCS(x,y)}{\max(len(x), len(y))}$

In [1]:
from bisect import bisect
import numpy as np

In [2]:
def LCS(x, y):
    ranges = []
    ranges.append(len(y))
    y_letters = list(y)
    for i in range(len(x)):
        positions = [j for j, l in enumerate(y_letters) if l == x[i]]
        positions.reverse()
        for p in positions:
            k = bisect(ranges, p)
            if k == bisect(ranges, p - 1):
                if k < len(ranges) - 1:
                    ranges[k] = p
                else:
                    ranges.insert(k, p)
    return len(ranges) - 1

In [3]:
a = "abcd"
b = "aabc"

In [4]:
LCS(a,b)

3

In [5]:
def LCS_metric(x,y):
    return 1 - LCS(x,y)/max(len(x), len(y))

In [6]:
LCS_metric(a,b)

0.25

### Utils

In [7]:
def ngrams(text,n):
    ng = {}
    for i in range(len(text) - n + 1):
        cur = text[i:i+n]
        if cur not in ng:
            ng[cur] = 0
            for j in range(len(text) - n + 1):
                if text[j:j+n] == cur:
                    ng[cur] += 1
    return ng

In [8]:
ngrams("alacala",3)

{'ala': 2, 'lac': 1, 'aca': 1, 'cal': 1}

In [66]:
def vectors(text1,text2,n=3):
    ng1 = ngrams(text1, n)
    ng2 = ngrams(text2, n)
    keys1 = set(ng1.keys())
    keys2 = set(ng2.keys())

    # join keys
    k = keys1 | keys2
    
    vecs1 = np.zeros(len(k))
    vecs2 = np.zeros(len(k))

    for idx,key in enumerate(k):
        vecs1[idx] = ng1.get(key,0)
        vecs2[idx] = ng2.get(key,0)
        
    return vecs1,vecs2

### Euclidean distance
$d(x,y) = \sqrt{\sum_{i=1}^{n}(x_i-y_i)^2}$

where $x,y$ are vectors

In [78]:
def Euclidean_metric(x,y,n=3):
    v1, v2 = vectors(x,y,n)
    dist = np.linalg.norm(v1 - v2)
    
    return dist

In [79]:
Euclidean_metric("alamakota","alapaala",3)

3.3166247903554

### Cosine distance
$d(x,y) = 1 - \frac{x \cdot y}{||x|| \cdot ||y||} $

In [80]:
def cosine_metric(x,y,n=3):
    v1, v2 = vectors(x,y,n)
    dist = 1 - np.dot(v1,v2)/np.linalg.norm(v1)/np.linalg.norm(v2)
    return dist

In [81]:
cosine_metric("alamakota","alapaala",3)

0.7327387580875757

### Zad 2.

### Davies-Bouldin index:
$DB=\frac{1}{n}\sum_{i=1}^{n}\max_{j \neq i} \frac{\sigma_i + \sigma_j}{d(c_i+c_j)}$

$c_x-$ centroid klastra

$\sigma_x-$średnia odległość między elementami klastra

### Zad 3.

In [71]:
def stoplist(text, frequency = 0.2):
    counted = {}
    for word in text.split():
        counted[word] = counted.get(word,0) + 1
    print(counted)
    
    stoplist = set()
    all_words = sum(counted.values())
    for word in counted:
        if counted[word]/all_words >= frequency:
            stoplist.add(word)
            
    return stoplist

In [72]:
t = "a a ab abc ab o q w e r"

In [73]:
stoplist(t)

{'a': 2, 'ab': 2, 'abc': 1, 'o': 1, 'q': 1, 'w': 1, 'e': 1, 'r': 1}


{'a', 'ab'}